<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)" data-toc-modified-id="Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description Task 1: RNN Language Modelling (30 +10 Points)</a></span><ul class="toc-item"><li><span><a href="#1a)-Language-Modelling-(30-Points)" data-toc-modified-id="1a)-Language-Modelling-(30-Points)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1a) Language Modelling (30 Points)</a></span></li><li><span><a href="#Conditional-Generation-(10-Points)" data-toc-modified-id="Conditional-Generation-(10-Points)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Conditional Generation (10 Points)</a></span></li></ul></li><li><span><a href="#Code-for-Task-1" data-toc-modified-id="Code-for-Task-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code for Task 1</a></span><ul class="toc-item"><li><span><a href="#Task-1.1" data-toc-modified-id="Task-1.1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Task 1.1</a></span><ul class="toc-item"><li><span><a href="#Setup-and-preparation" data-toc-modified-id="Setup-and-preparation-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Setup and preparation</a></span></li><li><span><a href="#Data-preprocessing" data-toc-modified-id="Data-preprocessing-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Data preprocessing</a></span></li><li><span><a href="#RNN" data-toc-modified-id="RNN-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>RNN</a></span><ul class="toc-item"><li><span><a href="#Dev:-playing-around-with-batches" data-toc-modified-id="Dev:-playing-around-with-batches-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Dev: playing around with batches</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Sample-and-test" data-toc-modified-id="Sample-and-test-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Sample and test</a></span></li></ul></li></ul></li></ul></div>

# Natural Language Understanding: Project 1

[__Natural Language Understanding, Spring 2018, ETHZ__](http://www.da.inf.ethz.ch/teaching/2018/NLU/)

[__Project 1__ (ETHZ network)](http://www.da.inf.ethz.ch/teaching/2018/NLU/material/project.pdf)

# Project to-do list:

Somewhat in order of importance:

- change code to unroll RNN in time instead of using dynamic_rnn
- ~~make sure the target data fed into the crossentropy metric is really in correct form~~
- ~~try own implementation of basic RNN cell instead of TF-prefab RNN or LSTM cell~~
- ~~change implementation to use the Xavier initializer instead of the uniform distribution currently used (see below)~~
- ~~change all `tf.Variable` variable inits to the better practice form like `W = tf.get_variable(name='example', shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())` which also includes the proper weight init~~
- ~~include dropout at input and/or RNN cell level for regularization~~
- ~~clean up namespaces, tensor naming~~
- ~~(Started 2.4.2018, but TBC)** build in all reporting for Tensorboard~~
- ~~adapt code to allow for differently sized timesteps~~
- ~~make arrangements to save trained model~~
- implement perplexity function
- ~~Maybe needs rewrite to use stock LSTM cell again** implement sampling function for conditional text generation~~
- ~~implement result output function~~
- adapt code to allow for use of pretrained word2vec embedding

## Description Task 1: RNN Language Modelling (30 +10 Points)

### 1a) Language Modelling (30 Points)
Your task is to build a simple LSTM language model. To be precise, we assume that words are independent given the recurrent hidden state; we compute a new hidden state given the last hidden state and last word, and predict the next word given the hidden state:
$$ P(w_1,\dots,w_n) = 􏰀\prod_{t=1}^{n}P(w_t|\mathbf{h}_t)$$
$$ P(w_t|\mathbf{h}_t) = \text{softmax}(\mathbf{Wh}_t)$$
$$ \mathbf{h}_t = f(\mathbf{h}_{t−1}, w_{t-1}^{*})$$

where $f$ is the LSTM recurrent function, $\mathbf{W} \in \mathbb{R}^{|V|×d}$ are softmax weights and $\mathbf{h_0}$ is either an all-zero constant or a trainable parameter.
You can use the tensorflow cell implementation __[1](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell)__ to carry out the recurrent computation in $f$. However, you must construct the actual RNN yourself (e.g. don’t use tensorflow’s `static_rnn` or `dynamic_rnn` or any other RNN library). That means, you will need to use a python loop that sets up the unrolled graph. To make your life simpler, please follow these design choices:

__Model and Data specification__

- Use a special sentence-beginning symbol `<bos>` and a sentence-end symbol `<eos>` (please use exactly these, including brackets). The `<bos>` symbol is the input, when predicting the first word and the `<eos>` symbol you require your model to predict at the end of every sentence.
- Use a maximum sentence length of 30 (including the `<bos>` and `<eos>` symbol). Ignore longer sentences during training and testing.
- Use a special padding symbol `<pad>` (please use exactly this, including brackets) to fill up sentences of length shorter than 30. This way, all your input will have the same size.
- Use a vocabulary consisting of the 20K most frequent words in the training set, including the symbols `<bos>`, `<eos>`, `<pad>` and `<unk>`. Replace out-of-vocabulary words with the `<unk>` symbol before feeding them into your network (don’t change the file content).
- Provide the ground truth last word as input to the RNN, not the last word you predicted. This is common practice.
- Language models are usually trained to minimize the cross-entropy. Use tensorflow’s `tf.nn.sparse_softmax_cross_entropy_with_logits` to compute the loss (*This operation fuses the computation of the soft-max and the cross entropy loss given the logits. For numerical stability, it’s very important to use this function.*). Use the AdamOptimizer with default parameters to minimize the loss. Use `tf.clip_by_global_norm` to clip the norm of the gradients to 5.
- Use a batch size of 64.
- Use the data at __[6](https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB)__. Don’t pre-process the input further. All the data is already white-space tokenized and
lower-cased. One sentence per line.
- To initialize your weight matrices, use the `tf.contrib.layers.xavier_initializer()` initializer introduced in __[5](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf)__.

__Experiments__
All experiments should not run for longer than, say, four hours on the GPU. For this task, your
grade won’t improve with performance.

- __Experiment A__: Train your model with word-embedding dimensionality of 100 and a hidden state size of 512 and compute sentence perplexity on the evaluation set (see submission format below).
- __Experiment B__: It is common practice, to pretrain word embeddings using e.g. `word2vec`. This should make your model train faster as words will come already with some useful representation. Use the code at __[3](http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py)__ to load these word embeddings __[4](https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U)__ trained on the same corpus. Train your model again and compute evaluation perplexity.
- __Experiment C__: It is often desirable to make the LSTM more powerful, by increasing the hidden dimensionality. However, this will naturally increase the parameters $\mathbf{W}$ of the softmax. As a compromise, one can use a larger hidden state, but down-project it before the softmax. Increase the hidden state dimensionality from 512 to 1024, but down-project $h_t$ to dimensionality 512 before predicting $w_t$ as in
$$ \mathbf{\tilde{h}}_t = \mathbf{W}_P\mathbf{h}_t$$
where $W_P$ are parameters. Train your model again and compute evaluation perplexity.

__Submission and grading__
- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - __Three__ result files containing sentence-level perplexity numbers on the __test__ set (to be distributed) for
all three experiments. Recall that perplexity of a sentence $S = ⟨w_1, \dots , w_n⟩$ with respect to your model $p(w_t|w_1, \dots, w_{t−1})$ is defined as
$$ \text{Perp} = 2^{-\frac{1}{n} \sum_{t=1}^{n}\log_2 p(w_t|w_1,\dots,w_{t−1})}$$
The `<eos>` symbol is part of the sequence, while the `<pad>` symbols (if any) are not. Be sure to have the basis of the exponential and the logarithm match.<br>
__Input format sentences.test__<br>
One sentence (none of them is longer than 28 tokens) per line:<br>
         ```beside her , jamie bounced on his seat .
         i looked and saw claire montgomery looking up at me .
         people might not know who alex was , but they knew to listen to him .```<br>
__Required output format groupXX.perplexityY__<br>
(where XX is your group __number__ and Y ∈ {A,B,C} is the experiment). One perplexity number per line:<br>
         $10.232$<br>
         $2.434$<br>
         $5.232$<br>
Make sure to have equally many lines in the output as there are in the input – otherwise your submission will be rejected automatically.
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018

### Conditional Generation (10 Points)
Let’s use your trained language model from above to generate sentences. Given an initial sequence of words, your are asked to __greedily__ generate words until either your model decides to finish the sentence (it generated `<eos>`) or a given maximum length has been reached. Note, that this task does not involve any training. Please see the tensorflow documentation on how to save and restore your model from above.
There are several ways how to implement the generation. For example, you can define a graph that computes just one step of the RNN given the last input and the last state (both from a new placeholder).
$$ \text{state}_t, p_t = f(\text{state}_{t−1},w_{t−1}) $$
That means, for a prefix of size $m$ and a desired length of $n$, you run this graph $n$ times. The first $m + 1$ times you take the input form the prefix. For the rest of the sequence, you take the most likely2 word $w^{t−1} = \text{argmax}_w p_{t−1}(w)$ from the last step.

- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - Your continued sentences of length up to 20. Use your trained model from experiment __C__ in task 1.1.
    __Input format sentences.continuation__ One sentence (of length less than 20) per line:<br>
         ```beside her ,
         i
         people might not know```<br>
    The `<bos>` symbol is not explicitly in the file, but you should still use it as the first input.<br>
    __Required output format groupXX.continuation__ (where XX is your __group number__)<br>
         ```beside her , something happened ! <eos>
         i do n’t recall making a noise , but i must have , because bob just looked up from his
         people might not know the answer . <eos>```
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018


__Infrastructure__

You must use Tensorflow, but any programming language is allowed. However, we strongly recommend `python3`. You have access to two compute resources: Unlimited CPU usage on Euler and GPU usage on Leonhard. Note that the difference in speed is typically a factor between 10 and 100.

## Code for Task 1
### Task 1.1

#### Setup and preparation

Make sure you have done the following:

- Download data from https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB and unpack into `./data/` subdirectory
- Download embeddings from https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U and unpack into `./data/` subdirectory
- Download helper function from http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py and put into `./helpers/` subdirectory

In [2]:
import tensorflow as tf
import numpy as np

import time

from collections import Counter
from helpers.load_embeddings import load_embedding
from tqdm import tqdm

%matplotlib inline

In [3]:
# Read all data from files

with open('./data/sentences.train', 'r') as f:
    train_data = f.read()
    
with open('./data/sentences.eval', 'r') as f:
    eval_data = f.read()
    
with open('./data/sentences.continuation', 'r') as f:
    continuation_data = f.read()    

In [4]:
# Have a peek at the given raw data

print('Training data sample:\n', 20*'=')
print(train_data[:100], '\n', 80*'.')

print('\n Evaluation data sample:\n', 20*'=')
print(eval_data[:100], '\n', 80*'.')

print('\n Continuation data sample:\n', 20*'=')
print(continuation_data[:100], '\n', 80*'.')

Training data sample:
`` i 've never had any ice-cream for myself , my father never let me have any . ''
`` and even if sh 
 ................................................................................

 Evaluation data sample:
he took my face in his hands and held my face where he wanted it as he kissed me senseless .
`` on t 
 ................................................................................

 Continuation data sample:
`` no ,
correct ...
`` i
peter
what was i
when he
take some of
id told
throw
we have no place
i felt 
 ................................................................................


#### Data preprocessing

In [5]:
# Splitting data into sentences
def split_data2sentences(data):
    text = ''.join(data)
    sentences = text.split('\n')
    return sentences

train_sentences = split_data2sentences(train_data)
eval_sentences = split_data2sentences(eval_data)
continuation_sentences = split_data2sentences(continuation_data)

## We generate the dictionary from the training data, so this treatment is different

# Get sentences from training data and look at sample
print('Sample training sentences:\n', train_sentences[:5], '\n')

# Make text contiguous again, break into words for vocabulary and look at sample
words = ' '.join(train_sentences).split()
print('Sample words:\n', words[:20], '\n')

# Make a word counter and show top frequency words
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
print('Top frequency words:\n', vocab[:20], '\n')

# Clip word counter to defined length [20k] and append special symbol words
symbols = ['<bos>', '<eos>', '<pad>', '<unk>']
vocab_length = 20000 # restrict to 20k most frequent tokens
vocab = vocab[:vocab_length-len(symbols)] # kick out the last 4 to replace w/ symbols
for each in symbols:
    vocab.append(each)
    
# Make a vocabulary to convert words to integers
vocab_to_int = {word: i for i, word in enumerate(vocab, 0)} # consider starting with 1 if 0 gives dead cells

# Make a vocabulary to get words from integers at the end
int_to_vocab = dict(enumerate(vocab))


Sample training sentences:
 ["`` i 've never had any ice-cream for myself , my father never let me have any . ''", "`` and even if she was , the gargoyle could hardly halt me from harming her . ''", 'pigafetta repeated the threat , which had no apparent effect .', "best to barricade oneself , no ? ''", "`` why would he lie ? ''"] 

Sample words:
 ['``', 'i', "'ve", 'never', 'had', 'any', 'ice-cream', 'for', 'myself', ',', 'my', 'father', 'never', 'let', 'me', 'have', 'any', '.', "''", '``'] 

Top frequency words:
 ['.', ',', 'the', 'i', 'to', 'and', '``', "''", 'a', 'he', 'of', 'you', 'was', 'her', 'it', 'she', 'in', 'his', 'that', '?'] 



In [6]:
# Encode sentences to integers and insert symbol words where necessary

### ATTN: Remove next line after finishing, keeping data set small for speedup
# train_sentences = train_sentences[:2000]

def encode_sentences(sentences):
    max_sentence_length = 30 # Given by task description
    sentences_ints = [] # List to hold converted-to-int sentences
    for each in tqdm(sentences):
        sentence = each.split()
        if len(sentence) < max_sentence_length-2: # -2 to allow for <bos>, <eos> \
                                                  # only use sentences w/ lenght <=30 as mandated
            sentence_int = [vocab_to_int['<bos>']] # Start sentence list w/ <bos>
            sentence_int += [vocab_to_int[word] if word in vocab_to_int\
                                 else vocab_to_int['<unk>'] for word in sentence] # Append remaining words
            sentence_int.append(vocab_to_int['<eos>']) # End sentence w/ <eos>
            while len(sentence_int) < max_sentence_length: # Pad length if necessary
                sentence_int.append(vocab_to_int['<pad>'])
            sentences_ints.append(sentence_int) 
    encoded = np.array(sentences_ints) # Convert list of sentences to np array
    return encoded

train_encoded = encode_sentences(train_sentences)
eval_encoded = encode_sentences(eval_sentences)
continuation_encoded = encode_sentences(continuation_sentences)

# Set data preparation complete flag
data_ready = True

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10001/10001 [00:00<00:00, 181708.54it/s]


#### RNN 

In [7]:
def variable_summaries(var):
    '''Attach a lot of summaries to a Tensor (for TensorBoard visualization).
    
        From TensorBoard documentation
    '''
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [14]:
class RNNLanguageModel:
    '''Main element: Class representing the complete RNN language model
    
        TBC
    '''
    
    def __init__(self,
                 out_size,
                 batch_size,
                 n_steps,
                 rnn_size,
                 rnn_size_factor,
                 rnn_layers,
                 learning_rate,
                 grad_clip,
                 embedding_size,
                 sampling=False):
        
        # Adapt batch_size, n_steps for sampling
        if (sampling == True):
             batch_size, n_steps = 1, 1
        
        # Reset tensorflow graph for clean slate
        tf.reset_default_graph()
        
        # Build input layer, keep_prob (all placeholders):
        with tf.name_scope('input_layer'):
            self.inputs = tf.placeholder(tf.int32, [batch_size, n_steps], name='inputs')
            self.targets = tf.placeholder(tf.int64, [batch_size, n_steps], name='targets')
            self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
        # Build embedding layer
        with tf.name_scope('embedding_layer'):
            embedding_matrix = tf.get_variable(name='embedding_matrix',
                                               shape=[vocab_length, embedding_size],
                                               initializer=unif)
            self.embeddings = tf.nn.embedding_lookup(embedding_matrix, self.inputs,
                                                     name='embeddings')
            
            tf.summary.histogram('embeddings', self.embeddings)
            tf.summary.scalar('dropout', keep_prob)
            
            # Embedding layer dropout during training
            if (keep_prob < 1):
                self.embeddings = tf.nn.dropout(self.embeddings, keep_prob, name='embeddings_dropout')
        
        # Build RNN layer
        with tf.name_scope('hidden_layer'):
            
            # LSTM cell with dropout wrapper
            cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size*rnn_size_factor)
            cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=keep_prob)
            
            # Initialize cell to zero state
            self.initial_state = cell.zero_state(batch_size, tf.float32)
    
            # LSTM time steps
            # Back to dynamic_rnn for the moment to make sure the rest works
            self.rnn_output, self.final_state = tf.nn.dynamic_rnn(cell, self.embeddings, initial_state=self.initial_state)

#             outputs = []
#             for i in range(n_steps-1):
#                 output, state = cell.(self.embeddings[:, i, :], state)
#                 outputs.append(output)
#             self.rnn_output, self.final_state = cell(tf.cast(self.targets, tf.float32), state)
            
            # to keep only last output would be:
            # self.rnn_output = self.rnn_output[:, -1, :]
        
            # Reshape hidden layer output: one row per input and step, i.e. ((batch_size*n_steps), rnn_size)            
            self.rnn_output = tf.reshape(self.rnn_output, [-1, rnn_size*rnn_size_factor])
            
            # If hidden layer increased by factor [1.1C], project down
            if (rnn_size_factor > 1):
                W_p = tf.get_variable(name='downprojection_weight', shape=[rnn_size*rnn_size_factor,
                                                                          rnn_size],
                                      initializer=xavi)
                b_p = tf.get_variable(name='downprojection_bias', shape=[rnn_size,],
                                      initializer=zeros)
                
                self.rnn_output = tf.nn.xw_plus_b(self.rnn_output, W_p, b_p)
                
                variable_summaries(W_p)
                variable_summaries(b_p)
                
        # Softmax output layer
        with tf.name_scope('softmax_layer'):
            # Hook up RNN outputs to softmax layer:
            W_softmax = tf.get_variable(name="softmax_weight", shape=[rnn_size, out_size],
                                        initializer=xavi)
            b_softmax = tf.get_variable(name="softmax_bias", shape=[out_size],
                                        initializer=zeros)
            
            variable_summaries(W_softmax)
            variable_summaries(b_softmax)
            
            # Calculate logits from softmax layer
            self.logits = tf.nn.xw_plus_b(self.rnn_output, W_softmax, b_softmax, name='logits')
            
            tf.summary.histogram('logits', self.logits)
            
            # Finally, get word probabilities from logits
            self.predictions = tf.nn.softmax(self.logits, name='predictions')
            
            tf.summary.histogram('predictions', self.predictions)
        
        # Loss and accuracy
        with tf.name_scope('loss_accuracy'):
            # One-hot encode targets:
            y_one_hot = tf.one_hot(self.targets, out_size)
            
            # Reshape encoded targets to fit logits
            y = tf.reshape(y_one_hot, self.logits.get_shape())

            # Softmax cross entropy loss
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,
                                                                               labels=y))
            
            tf.summary.scalar('loss', self.loss)

            # Best prediction
            self.best_prediction = tf.argmax(self.predictions, 1, name='best_prediction')
            
            # Accuracy
            correct_prediction = tf.equal(self.best_prediction, tf.argmax(y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
            tf.summary.scalar('accuracy', self.accuracy)

        # Optimizer
        with tf.name_scope('optimizer'):
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), grad_clip)
            train_op = tf.train.AdamOptimizer(learning_rate)
            self.optimizer = train_op.apply_gradients(zip(grads, tvars))      

        # Merge summaries for TensorBoard    
        self.merged = tf.summary.merge_all()

In [9]:
class RNNConfig(object):
    '''Class holding all configuration vars for training the RNN
    
    '''
    
    def __init__(self,
                 train_data__path='./data/sentences.train',
                 eval_data__path='./data/sentences.eval',
                 validation_split=.1,
                 vocab_length=20000,
                 embedding_size=100,
                 rnn_size=512,
                 rnn_size_factor=1,
                 rnn_layers=1,
                 n_steps=10,
                 learning_rate=0.001,
                 keep_prob=0.5,
                 grad_clip=5.0,
                 batch_size=64,
                 num_epochs=10,
                 save_every_n=500,
                 validate_every_n=500,
                 report_every_n=100,
                 summary_every_n=10,
                 max_to_keep=5,
                 past_words=5):
        
        # Path to the training data 
        self.train_data__path = train_data__path
        
        # Path to the evaluation data 
        self.eval_data__path = eval_data__path
        
        # Percentage of the training data used for validation (default: 10%)
        self.validation_split = validation_split
        
        # Size of the vocabulary (default: 20k)
        self.vocab_length = vocab_length
        
        # Dimensionality of word embedding layer (default: 100)
        self.embedding_size = embedding_size
        
        # Dimensionality of RNN (i.e. hidden) layer (default: 512)
        self.rnn_size = rnn_size
        
        # Integer to factor the size of the hidden layer [Task 1.1C] (default: 1)
        self.rnn_size_factor = rnn_size_factor
        
        # Number of RNN layers (default: 1)
        self.rnn_layers = rnn_layers
        
        # Number of time steps for RNN (default: 30)
        self.n_steps = n_steps
        
        # Size of softmax output (default: vocab_length)
        self.out_size = self.vocab_length
        
        # Learning rate (default: 0.001)
        self.learning_rate = learning_rate
        
        # Dropout rate (default: 0.5)
        self.keep_prob = keep_prob
        
        # Gradient clipping treshold (default: 5.0)
        self.grad_clip = grad_clip
        
        # Batch Size (default: 64)
        self.batch_size = batch_size
        
        # Number of training epochs (default: 10)
        self.num_epochs = num_epochs
        
        # Save model after this many steps (default: 500)
        self.save_every_n = save_every_n
        
        # Validate after this many steps (default: 500)
        self.validate_every_n = validate_every_n
        
        # Print report after this many steps (default: 100)
        self.report_every_n = report_every_n
        
        # Record summary after this many steps (default: 10)
        self.summary_every_n = summary_every_n
        
        # Number of checkpoints to save (default: 5)
        self.max_to_keep = max_to_keep
        
        # How many previous words are used for prediction (default: 5)
        self.past_words = past_words

In [10]:
# Build batch generator for training    
def get_batches(source_arr, batch_size, n_steps):
    '''Generator which returns features x and targets y
    
    Inputs
    ------
    source_arr: A np.ndarray of sentences in rows to generate features and targets from
    batch_size: An int number of sequences required per batch
    n_steps: Number of time steps for RNN to consider; defines sequence length
    
    Outputs
    -------
    x: A np.ndarray of feature sequences according to parameters above
    y: A np.ndarray of target sequences according to paramteres above.
    '''
    
    source_rows = source_arr.shape[0]
    source_cols = source_arr.shape[1]
    
    # Make sure time steps doesn't exceed available information in sentence
    assert (n_steps <  source_cols), "Can't look further back than sentence length"
    
    # How many sequences we can get from an input sentence (row)
    n_seq_per_row = source_cols - n_steps 
        
    # How many 'batch blocks' we can get
    n_blocks = source_rows // batch_size
   
    # Shuffle rows of sequences to improve training
    np.random.shuffle(source_arr)
    
    # Crop array to only produce full batches
    source_arr = source_arr[:n_blocks*batch_size, :]

    # Reshape source_arr for easier batch generation
    source_arr = source_arr.reshape((batch_size, -1))
    
    # Generate batches
    for j in range(0, source_arr.shape[1], source_cols):
        for jj in range(n_seq_per_row):
            # Feature sequence:
            x = source_arr[:, j+jj  :j+jj+n_steps  ]
            # Target:
            y = source_arr[:, j+jj+1:j+jj+n_steps+1]
            yield x, y

In [ ]:
# Development only: playing around with batches

shuffled_rows_ind = np.random.permutation(len(train_encoded))
validation_split_ind = int(0.1 * len(train_encoded))
source_train = train_encoded[shuffled_rows_ind[validation_split_ind:]]
batches = get_batches(source_train, 3, 8)

x, y = next(batches)
print('x\n',x.shape,'\n', x)
print('y\n',y.shape,'\n', y)
for i in range(x.shape[0]):
        print([int_to_vocab[x[i,j]] for j in range(x[i].shape[0])], ' -> ', [int_to_vocab[y[i,j]] for j in range(y[i].shape[0])])

#### Training

In [ ]:
# Define shorthands for common initializers
ones = tf.ones_initializer
unif = tf.random_uniform_initializer(-1, 1)
xavi = tf.contrib.layers.xavier_initializer()
zeros = tf.zeros_initializer()

# Set configuration for training
CONFIG = RNNConfig(num_epochs=1 , report_every_n=50, validate_every_n=100, rnn_size_factor=1, n_steps=15)

# Check data preparation complete
assert data_ready == True, 'Need to run data preparation first'

# Split training/validation data
np.random.seed(42)
shuffled_rows_ind = np.random.permutation(len(train_encoded))
validation_split_ind = int(CONFIG.validation_split * len(train_encoded))
source_train = train_encoded[shuffled_rows_ind[validation_split_ind:]]
source_validation = train_encoded[shuffled_rows_ind[:validation_split_ind]]

# Set training loop variables
num_epochs = CONFIG.num_epochs
save_every_n = CONFIG.save_every_n
validate_every_n = CONFIG.validate_every_n
report_every_n = CONFIG.report_every_n
summary_every_n = CONFIG.summary_every_n
keep_prob = CONFIG.keep_prob

# Create model instance
model = RNNLanguageModel(out_size=CONFIG.out_size,
                         batch_size=CONFIG.batch_size,
                         n_steps=CONFIG.n_steps,
                         rnn_size=CONFIG.rnn_size,
                         rnn_size_factor=CONFIG.rnn_size_factor,
                         rnn_layers=CONFIG.rnn_layers,
                         learning_rate=CONFIG.learning_rate,
                         grad_clip=CONFIG.grad_clip,
                         embedding_size=CONFIG.embedding_size)

# Setup TensorBoard logging
now = time.strftime('%y-%m-%d-%H-%M-%S')
log_dir = './runs'
log_subdir = '{}/run-{}/'.format(log_dir, now)
writer = tf.summary.FileWriter(log_subdir, tf.get_default_graph())

# Setup model saving
save_dir = './checkpoints'
saver = tf.train.Saver(max_to_keep=CONFIG.max_to_keep)

# Run training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    
    for ep in range(num_epochs):
        new_state = sess.run(model.initial_state)
        loss = 0
        
        validation_batches = get_batches(source_validation, batch_size=CONFIG.batch_size,
                                n_steps=CONFIG.n_steps)
        
        for x_train, y_train in get_batches(source_train, batch_size=CONFIG.batch_size,
                                n_steps=CONFIG.n_steps):
            counter += 1
            start = time.time()
            
            feed_dict = {model.inputs: x_train,
                         model.targets: y_train,
                         model.keep_prob: keep_prob,
                         model.initial_state: new_state}
            
            loss, accuracy, best_prediction, new_state, _ = \
            sess.run([model.loss, model.accuracy, model.best_prediction, model.final_state,
                      model.optimizer], feed_dict=feed_dict)
            
            batch_time = time.time() - start
            
            if (counter % summary_every_n == 0):
                writer.add_summary(sess.run(model.merged, feed_dict=feed_dict), counter)
            
            if (counter % report_every_n == 0):
                print('Epoch: {}/{}\t'.format(ep+1, num_epochs),
                      'Training step: {}\t'.format(counter),
                      'Batch loss: {:.4f}\t'.format(loss),
                      'Batch accuracy: {:.1%}\t'.format(accuracy),
                      '{:4f} sec/batch'.format(batch_time))
                print(' '.join([int_to_vocab[x_train[0,j]] for j in range(x_train[0].shape[0])]),
                      ' -> ', int_to_vocab[best_prediction[0]] , ' vs. ', int_to_vocab[y_train[0, -1]])
            
            if (counter % save_every_n == 0):
                saver.save(sess, '{}/run-{}_i{}_s{}.ckpt'.format(save_dir, now,
                                                                 counter, CONFIG.rnn_size))
            if (counter % validate_every_n == 0):
                x_validation, y_validation = next(validation_batches)
                feed_dict = {model.inputs: x_validation,
                             model.targets: y_validation,
                             model.keep_prob: 1.0,
                             model.initial_state: new_state}
                
                loss, accuracy = sess.run([model.loss, model.accuracy], feed_dict=feed_dict)
                print('Epoch: {}/{}\t'.format(ep+1, num_epochs),
                      '*** Validation ***\t',
                      'Loss: {:.4f}'.format(loss),
                      'Accuracy: {:.1%}\t'.format(accuracy))
                
    writer.close()
    saver.save(sess, '{}/run-{}_i{}_s{}.ckpt'.format(save_dir, now, counter, CONFIG.rnn_size))

In [16]:
# Development only: Check trainable variables

tf.trainable_variables()

[<tf.Variable 'embedding_matrix:0' shape=(20000, 100) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(612, 2048) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(2048,) dtype=float32_ref>,
 <tf.Variable 'softmax_weight:0' shape=(512, 20000) dtype=float32_ref>,
 <tf.Variable 'softmax_bias:0' shape=(20000,) dtype=float32_ref>]

#### Sample and test

In [12]:
def generate_samples(checkpoint, n_samples, primer):
    samples = [w for w in primer.split()]
    primer_feed = [vocab_to_int['<bos>']]
    [primer_feed.append(vocab_to_int[word]) for word in samples]
    model = RNNLanguageModel(out_size=CONFIG.out_size,
                             batch_size=CONFIG.batch_size,
                             n_steps=CONFIG.n_steps,
                             rnn_size=CONFIG.rnn_size,
                             rnn_size_factor=CONFIG.rnn_size_factor,
                             rnn_layers=CONFIG.rnn_layers,
                             learning_rate=CONFIG.learning_rate,
                             grad_clip=CONFIG.grad_clip,
                             embedding_size=CONFIG.embedding_size,
                             sampling=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for i in primer_feed:
            x = np.zeros((1,1))
            x[0, 0] = i
            feed_dict = {model.inputs: x,
                         model.keep_prob: 1.0,
                         model.initial_state: new_state}
            best_prediction, new_state = \
            sess.run([model.best_prediction, model.final_state], feed_dict=feed_dict)
            
        samples.append(int_to_vocab[best_prediction[0]])
        for j in range(n_samples):
            x[0, 0] = best_prediction
            feed_dict = {model.inputs: x,
                         model.keep_prob: 1.0,
                         model.initial_state: new_state}
            best_prediction, new_state = \
            sess.run([model.best_prediction, model.final_state], feed_dict=feed_dict)
            samples.append(int_to_vocab[best_prediction[0]])
            if (samples[-1] == '<eos>'):
                break
    return ' '.join(samples)

In [13]:
# Get latest checkpoint
checkpoint = tf.train.latest_checkpoint(save_dir)
# checkpoint = './checkpoints/run-18-04-07-18-21-43_i19000_s512.ckpt'

# Sentence generation length limit
max_generate_n = 20

# Generate some text
for i in range(10, 1000, 27):
    output = generate_samples(checkpoint, max_generate_n, continuation_sentences[i])
    print(continuation_sentences[i], ' -> ', output)

INFO:tensorflow:Restoring parameters from ./checkpoints\run-18-04-07-21-52-52_i2500_s512.ckpt
i felt that hot  ->  i felt that hot , but i was n't know the <unk> , but i was n't a few . <eos>
INFO:tensorflow:Restoring parameters from ./checkpoints\run-18-04-07-21-52-52_i2500_s512.ckpt
`` what do  ->  `` what do n't know you ? '' <eos>
INFO:tensorflow:Restoring parameters from ./checkpoints\run-18-04-07-21-52-52_i2500_s512.ckpt
`` i have a  ->  `` i have a few seats . <eos>
INFO:tensorflow:Restoring parameters from ./checkpoints\run-18-04-07-21-52-52_i2500_s512.ckpt
thats where  ->  thats where i was n't been been a <unk> . <eos>
INFO:tensorflow:Restoring parameters from ./checkpoints\run-18-04-07-21-52-52_i2500_s512.ckpt
``  ->  `` you was going to make the <unk> , but i was n't been to me . <eos>
INFO:tensorflow:Restoring parameters from ./checkpoints\run-18-04-07-21-52-52_i2500_s512.ckpt
`` good luck  ->  `` good luck to you . <eos>
INFO:tensorflow:Restoring parameters from ./checkpo